In [1]:
import nltk
import re 
from natasha import (
    Segmenter,
    MorphVocab,
    NewsEmbedding,
    NewsMorphTagger,
    Doc
)
segmenter = Segmenter()  
morph_vocab = MorphVocab()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()

у нас не получилось почистить текст, собранный парсером, поэтому мы решили собрать корпус (отчасти) вручную. судя по формулировке 1 пункта, так тоже можно, и без снижения баллов. ниже код, с помощью которого мы создавали корпус. еще чуть ниже — полученный корпус :)

In [2]:
'''def str_split(str):
    cents = nltk.tokenize.sent_tokenize(strg)
    return cents


corpus = []
x = 0
while input('stop?') == '': 
    name = input('название:')
    strg = input('текст:')
    text = str_split(strg)
    corpus.append((name, text))
    x += 1
    print(x)
    corpus.append((name, text))
print(corpus)'''

"def str_split(str):\n    cents = nltk.tokenize.sent_tokenize(strg)\n    return cents\n\n\ncorpus = []\nx = 0\nwhile input('stop?') == '': \n    name = input('название:')\n    strg = input('текст:')\n    text = str_split(strg)\n    corpus.append((name, text))\n    x += 1\n    print(x)\n    corpus.append((name, text))\nprint(corpus)"

In [3]:
corpus = [('«Агафья»', ['Случилось мне быть у этого самого Савки в один из хороших майских вечеров.', 'Помню, я лежал на рваной, затасканной полости почти у самого шалаша, от которого шел густой и душный запах сухих трав.', 'Подложив руки под голову, я глядел вперед себя.', 'У ног моих лежали деревянные вилы.', 'За ними черным пятном резалась в глаза собачонка Савки — Кутька, а не дальше, как сажени на две от Кутьки, земля обрывалась в крутой берег речки.', 'Лежа я не мог видеть реки.', 'Я видел только верхушки лозняка, теснившегося на этом берегу, да извилистый, словно обгрызенный край противоположного берега.', 'Далеко за берегом, на темном бугре, как испуганные молодые куропатки, жались друг к другу избы деревни, в которой жил мой Савка.', 'За бугром догорала вечерняя заря.', 'Осталась одна только бледно-багровая полоска, да и та стала подергиваться мелкими облачками, как уголья пеплом.']), ('«Анна на шее»', ['Пока она вспоминала эти подробности, послышалась вдруг музыка, ворвавшаяся в окно вместе с шумом голосов.', 'Это поезд остановился на полустанке.', 'За платформой в толпе бойко играли на гармонике и на дешевой визгливой скрипке, а из-за высоких берез и тополей, из-за дач, залитых лунным светом, доносились звуки военного оркестра: должно быть, на дачах был танцевальный вечер.', 'На платформе гуляли дачники и горожане, приезжавшие сюда в хорошую погоду подышать чистым воздухом.', 'Был тут и Артынов, владелец всего этого дачного места, богач, высокий, полный брюнет, похожий лицом на армянина, с глазами навыкате и в странном костюме.', 'На нем была рубаха, расстегнутая на груди, и высокие сапоги со шпорами, и с плеч спускался черный плащ, тащившийся по земле, как шлейф.', 'За ним, опустив свои острые морды, ходили две борзые.']), ('«Ариадна»', ['И сама Ариадна знала, что я ее люблю.', 'Она часто приезжала к нам верхом или на шарабане и проводила иногда целые дни со мною и с отцом.', 'С моим стариком она подружилась, и он даже научил ее кататься на велосипеде — это было его любимое развлечение.', 'Помню, как однажды вечером они собрались кататься и я помогал ей сесть на велосипед, и в это время она была так хороша, что мне казалось, будто я, прикасаясь к ней, обжигал себе руки, я дрожал от восторга, и когда они оба, старик и она, красивые, стройные, покатили рядом по шоссе, встречная вороная лошадь, на которой ехал приказчик, бросилась в сторону, и мне показалось, что она бросилась оттого, что была тоже поражена красотой.', 'Моя любовь, мое поклонение трогали Ариадну, умиляли ее, и ей страстно хотелось быть тоже очарованною, как я, и отвечать мне тоже любовью.', 'Ведь это так поэтично!']), ('«Архиерей»', ['Под вербное воскресение в Старо-Петровском монастыре шла всенощная.', 'Когда стали раздавать вербы, то был уже десятый час на исходе, огни потускнели, фитили нагорели, было всё, как в тумане.', 'В церковных сумерках толпа колыхалась, как море, и преосвященному Петру, который был нездоров уже дня три, казалось, что все лица — и старые, и молодые, и мужские, и женские — походили одно на другое, у всех, кто подходил за вербой, одинаковое выражение глаз.', 'В тумане не было видно дверей, толпа всё двигалась, и похоже было, что ей нет и не будет конца.', 'Пел женский хор, канон читала монашенка.']), ('«Бабы»', ['Поздно вечером, когда уже спали и Дюдя, и старуха, и соседний сторож, Софья вышла за ворота и села на лавочку.', 'Ей было душно, и от слез разболелась голова.', 'Улица была широкая и длинная; направо версты две, налево столько же, и конца ей не видно.', 'Луна уже ушла от двора и стояла за церковью.', 'Одна сторона улицы была залита лунным светом, а другая чернела от теней; длинные тени тополей и скворешен тянулись через всю улицу, а тень от церкви, черпая и страшная, легла широко и захватила ворота Дюди и половину дома.', 'Было безлюдно и тихо.', 'С конца улицы изредка доносилась едва слышная музыка; должно быть, это Алёшка играл на своей гармонике.']), ('«Барыня»', ['Как только отъехала коляска и скрылась из вида, в окне показался Степан.', 'Бледный, как смерть, дрожащий, он выполз наполовину из окна и погрозил своим большим кулаком темневшему вдали саду.', 'Сад был барский.', 'Погрозив раз шесть, он проворчал что-то, потянулся назад в избу и с шумом опустил раму.', 'Через полчаса после того, как уехала барыня, в избе Журкина ужинали.', 'В кухне возле самой печи за засаленным столом сидели Журкин и его жена.', 'Против них сидел старший сын Максима — Семен, временноотпускной, с красным испитым лицом, длинным рябым носом и маслеными глазками.', 'Семен был похож лицом на отца, он не был только сед, лыс и не имел таких хитрых, цыганских глаз, какими обладал его отец.', 'Рядом с Семеном сидел второй сын Максима, Степан.', 'Степан не ел, а, подперевши кулаком свою красивую белокурую голову, смотрел на закопченный потолок и о чем-то усердно мыслил.', 'Ужин подавала жена Степана, Марья.', 'Щи съели молча.']), ('«Беглец»', ['Это была длинная процедура.', 'Сначала Пашка шел с матерью под дождем то по скошенному полю, то по лесным тропинкам, где к его сапогам липли желтые листья, шел до тех пор, пока не рассвело.', 'Потом он часа два стоял в темных сенях и ждал, когда отопрут дверь.', 'В сенях было не так холодно и сыро, как на дворе, по при ветре и сюда залетали дождевые брызги.', 'Когда сени мало-помалу битком набились народом, стиснутый Пашка припал лицом к чьему-то тулупу, от которого сильно пахло соленой рыбой, и вздремнул.', 'Но вот щелкнула задвижка, дверь распахнулась, и Пашка с матерью вошел в приемную.', 'Тут опять пришлось долго ждать.', 'Все больные сидели на скамьях, не шевелились и молчали.', 'Пашка оглядывал их и тоже молчал, хотя видел много странного и смешного.', 'Раз только, когда в приемную, подпрыгивая на одной ноге, вошел какой-то парень, Пашке самому захотелось также попрыгать.']), ('«Беда», 1886', ['С Николаем Максимычем Путохиным приключилась беда, от которой широким и беспечным российским натурам так же не следует зарекаться, как от тюрьмы и сумы: он невзначай напился пьян и в пьяном образе, забыв про семью и службу, ровно пять дней и ночей шатался по злачным местам.', 'От этих беспутно проведенных пяти суток в его памяти уцелел один только похожий на кашу сумбур из пьяных рож, цветных юбок, бутылок, дрыгающих ног.', 'Он напрягал свою память, и для него ясно было только, как вечером, когда зажигали фонари, он забежал на минутку к своему приятелю поговорить о деле, как приятель предложил выпить пива... Путохин выпил стакан, другой, третий... После шести бутылок приятели отправились к какому-то Павлу Семеновичу; этот последний угостил копченым сигом и мадерой.']), ('«Беззащитное существо»', ['— Изволите ли видеть, ваше превосходительство, — начала скороговоркой просительница, — муж мой, коллежский асессор Щукин, проболел пять месяцев, и, пока он, извините, лежал дома и лечился, ему без всякой причины отставку дали, ваше превосходительство, а когда я пошла за его жалованьем, они, изволите видеть, вычли из его жалованья 24 рубля 36 коп.!', 'За что?', '— спрашиваю.', '— «А он, говорят, из товарищеской кассы брал и за него другие чиновники ручались».', 'Как же так?', 'Нешто он мог без моего согласия брать?', 'Это невозможно, ваше превосходительство.', 'Да почему такое?', 'Я женщина бедная, только и кормлюсь жильцами... Я слабая, беззащитная... От всех обиду терплю и ни от кого доброго слова не слышу...']), ('«Белолобый»', ['В верстах четырех от ее логовища, у почтовой дороги, стояло зимовье.', 'Тут жил сторож Игнат, старик лет семидесяти, который все кашлял и разговаривал сам с собой; обыкновенно ночью он спал, а днем бродил по лесу с ружьем-одностволкой и посвистывал на зайцев.', 'Должно быть, раньше он служил в механиках, потому что каждый раз, прежде чем остановиться, кричал себе: «Стоп, машина!» и прежде чем пойти дальше: «Полный ход!» При нем находилась громадная черная собака неизвестной породы, по имени Арапка.', 'Когда она забегала далеко вперед, то он кричал ей: «Задний ход!» Иногда он пел и при этом сильно шатался и часто падал (волчиха думала, что это от ветра) и кричал: «Сошел с рельсов!»']), ('«Беседа пьяного с трезвым чёртом»', ['Бывший чиновник интендантского управления, отставной коллежский секретарь Лахматов, сидел у себя за столом и, выпивая шестнадцатую рюмку, размышлял о братстве, равенстве и свободе.', 'Вдруг из-за лампы выглянул на него чёрт... Но не пугайтесь, читательница.', 'Вы знаете, что такое чёрт?', 'Это молодой человек приятной наружности, с черной, как сапоги, рожей и с красными выразительными глазами.', 'На голове у него, хотя он и не женат, рожки... Прическа à la Капуль.', 'Тело покрыто зеленой шерстью и пахнет псиной.', 'Внизу спины болтается хвост, оканчивающийся стрелой... Вместо пальцев — когти, вместо ног — лошадиные копыта.', 'Лахматов, увидев чёрта, несколько смутился, но потом, вспомнив, что зеленые черти имеют глупое обыкновение являться ко всем вообще подвыпившим людям, скоро успокоился.']), ('«В вагоне»', ['Почтовый поезд номер такой-то мчится на всех парах от станции «Веселый Трах-Тарарах» до станции «Спасайся, кто может!».', 'Локомотив свистит, шипит, пыхтит, сопит... Вагоны дрожат и своими неподмазанными колесами воют волками и кричат совами.', 'На небе, на земле и в вагонах тьма... «Что-то будет!', 'что-то будет!» — стучат дрожащие от старости лет вагоны... «Огого-гого-о-о!» — подхватывает локомотив... По вагонам вместе с карманолюбцами гуляют сквозные ветры.', 'Страшно... Я высовываю свою голову в окно и бесцельно смотрю в бесконечную даль.', 'Все огни зеленые: скандал, надо полагать, еще не скоро.', 'Диска и станционных огней не видно... Тьма, тоска, мысль о смерти, воспоминания детства... Боже мой!']), ('«В овраге»', ['Как-то на поминках у фабриканта Костюкова старик-дьячок увидел среди закусок зернистую икру и стал есть ее с жадностью; его толкали, дергали за рукав, но он словно окоченел от наслаждения: ничего не чувствовал и только ел.', 'Съел всю икру, а в банке было фунта четыре.', 'И прошло уж много времени с тех пор, дьячок давно умер, а про икру всё помнили.', 'Жизнь ли была так бедна здесь, или люди не умели подметить ничего, кроме этого неважного события, происшедшего десять лет назад, а только про село Уклеево ничего другого не рассказывали.']), ('«В потемках»', ['Муха средней величины забралась в нос товарища прокурора, надворного советника Гагина.', 'Любопытство ли ее мучило, или, быть может, она попала туда по легкомыслию, или благодаря потемкам, но только нос не вынес присутствия инородного тела и подал сигнал к чиханию.', 'Гагин чихнул, чихнул с чувством, с пронзительным присвистом и так громко, что кровать вздрогнула и издала звук потревоженной пружины.', 'Супруга Гагина, Марья Михайловна, крупная, полная блондинка, тоже вздрогнула и проснулась.', 'Она поглядела в потемки, вздохнула и повернулась на другой бок.', 'Минут через пять она еще раз повернулась, закрыла плотнее глаза, но сон уже не возвращался к ней.', 'Повздыхав и поворочавшись с боку на бок, она приподнялась, перелезла через мужа и, надев туфли, пошла к окну.']), ('«В ссылке»', ['Старый Семен, прозванный Толковым, и молодой татарин, которого никто не знал по имени, сидели на берегу около костра; остальные три перевозчика находились в избе.', 'Семен, старик лет шестидесяти, худощавый и беззубый, но широкий в плечах и на вид еще здоровый, был пьян; он давно бы уже пошел спать, но в кармане у него был полуштоф, и он боялся, как бы в избе молодцы не попросили у него водки.', 'Татарин был болен, томился и, кутаясь в свои лохмотья, рассказывал, как хорошо в Симбирской губернии и какая у него осталась дома красивая и умная жена.', 'Ему было лет двадцать пять, не больше, а теперь, при свете костра, он, бледный, с печальным болезненным лицом, казался мальчиком.']), ('«Ванька»', ['Ванька Жуков, девятилетний мальчик, отданный три месяца тому назад в ученье к сапожнику Аляхину, в ночь под Рождество не ложился спать.', 'Дождавшись, когда хозяева и подмастерья ушли к заутрене, он достал из хозяйского шкапа пузырек с чернилами, ручку с заржавленным пером и, разложив перед собой измятый лист бумаги, стал писать.', 'Прежде чем вывести первую букву, он несколько раз пугливо оглянулся на двери и окна, покосился на темный образ, по обе стороны которого тянулись полки с колодками, и прерывисто вздохнул.', 'Бумага лежала на скамье, а сам он стоял перед скамьей на коленях.', '«Милый дедушка, Константин Макарыч!', '— писал он.', '— И пишу тебе письмо.', 'Поздравляю вас с Рождеством и желаю тебе всего от господа бога.', 'Нету у меня ни отца, ни маменьки, только ты у меня один остался».']), ('«Ведьма»', ['— Ведьма и есть ведьма, — продолжал Савелий глухим, плачущим голосом, торопливо сморкаясь в подол рубахи.', '— Хоть ты и жена мне, хоть и духовного звания, но я о тебе и на духу так скажу, какая ты есть... Да как же?', 'Заступи, господи, и помилуй!', 'В прошлом годе под пророка Даниила и трех отроков была метель и — что же?', 'мастер греться заехал.', 'Потом на Алексея, божьего человека, реку взломало, и урядника принесло... Всю ночь тут с тобой, проклятый, калякал, а как наутро вышел, да как взглянул я на него, так у него под глазами круги и все щеки втянуло!', 'А?', 'В Спасовку два раза гроза была, и в оба разы охотник ночевать приходил.', 'Я всё видел, чтоб ему пропасть!', 'Всё!', 'О, красней рака стала!', 'Ага!']), ('«Верочка»', ['— Привык я к вам, как легавый!', '— продолжал Огнев.', '— Почти каждый день к вам шлялся, раз десять ночевал, а наливки выпил столько, что теперь вспоминать страшно.', 'А главное, за что спасибо, Гавриил Петрович, так это за ваше содействие и помощь.', 'Без вас я со своей статистикой до октября бы тут возился.', 'Так и напишу в предисловии: считаю долгом выразить мою благодарность председателю N-ской уездной земской управы Кузнецову за его любезное содействие.', 'У статистики бле-естящая будущность!', 'Вере Гавриловне нижайший поклон, а докторам, обоим следователям и вашему секретарю передайте, что никогда не забуду их помощи!', 'А теперь, старче, обымем друг друга и сотворим последнее лобзание.']), ('«Винт»', ['И, не желая долее оставаться в недоумении, Пересолин открыл дверь и вошел в дежурную.', 'Если бы перед чиновниками явился сам чёрт с рогами и с хвостом, то он не удивил бы и не испугал так, как испугал и удивил их начальник.', 'Явись перед ними умерший в прошлом году экзекутор, проговори он им гробовым голосом: «Идите за мной, аггелы, в место, уготованное канальям», и дыхни он на них холодом могилы, они не побледнели бы так, как побледнели, узнав Пересолина.', 'У Недоехова от перепугу даже кровь из носа пошла, а у Кулакевича забарабанило в правом ухе и сам собою развязался галстук.', 'Чиновники побросали карты, медленно поднялись и, переглянувшись, устремили свои взоры на пол.', 'Минуту в дежурной царила тишина...']), ('«Водевиль»', ['— Да, немножко резко, — согласился Замазурин.', '— Ты как-нибудь замажь это место... Выкинь также рассуждение про то, как приятно быть тестем начальника.', 'Приятно, а ты смеешься... Этим, брат, шутить нельзя... Наш тоже на бедной женился, так из этого следует, что он скверно поступил?', 'Так, по-твоему?', 'Нешто ему не обидно?', 'Ну, положим, он сидит в театре и видит это самое... Нешто ему приятно?', 'А ведь он же твою руку держал, когда ты с Салалеевым пособия просил!', '«Он, говорит, человек больной, ему, говорит, деньги нужней, чем Салалееву»... Видишь?', '— А ты ведь, признайся, здесь на него намекаешь!', '— мигнул глазом Булягин.', '— И не думал!', '— сказал Клочков.', '— Накажи меня бог, совсем ни на кого не намекал!']), ('«Восклицательный знак»', ['«Сорок лет служил, — думал он, — и никто меня дураком не назвал, а тут, поди ты, какие критики нашлись!', '„Бессознательно!..', 'Лефректорно!', 'Машинное производство“... Ах, ты, чёрт тебя возьми!', 'Да я еще, может быть, больше тебя понимаю, даром что в твоих университетах не был!» Излив мысленно по адресу критика все известные ему ругательства и согревшись под одеялом, Перекладин стал успокаиваться.', '«Я знаю... понимаю... — думал он, засыпая.', '— Не поставлю там двоеточия, где запятую нужно, стало быть, сознаю, понимаю.', 'Да... Так-то, молодой человек... Сначала пожить нужно, послужить, а потом уж стариков судить...» В закрытых глазах засыпавшего Перекладина сквозь толпу темных, улыбавшихся облаков метеором пролетела огненная запятая.', 'За ней другая, третья, и скоро весь безграничный темный фон, расстилавшийся перед его воображением, покрылся густыми толпами летавших запятых...']), ('«Враги»', ['В десятом часу темного сентябрьского вечера у земского доктора Кирилова скончался от дифтерита его единственный сын, шестилетний Андрей.', 'Когда докторша опустилась на колени перед кроваткой умершего ребенка и ею овладел первый приступ отчаяния, в передней резко прозвучал звонок.', 'По случаю дифтерита вся прислуга еще с утра была выслана из дому.', 'Кирилов, как был, без сюртука, в расстегнутой жилетке, не вытирая мокрого лица и рук, обожженных карболкой, пошел сам отворять дверь.', 'В передней было темно, и в человеке, который вошел, можно было различить только средний рост, белое кашне и большое, чрезвычайно бледное лицо, такое бледное, что, казалось, от появления этого лица в передней стало светлее...']), ('«Выигрышный билет»', ['Иван Дмитрич не верил в лотерейное счастие и в другое время ни за что не стал бы глядеть в таблицу тиражей, но теперь от нечего делать и — благо, газета была перед глазами — он провел пальцем сверху вниз по номерам серий.', 'И тотчас же, точно в насмешку над его неверием, не дальше как во второй строке сверху резко бросилась в глаза цифра 9 499!', 'Не поглядев, какой номер билета, не проверяя себя, он быстро опустил газету на колени и, как будто кто плеснул ему на живот холодной водой, почувствовал под ложечкой приятный холодок: и щекотно, и страшно, и сладко!']), ('«Горе»', ['Токарь Григорий Петров, издавна известный за великолепного мастера и в то же время за самого непутевого мужика во всей Галчинской волости, везет свою больную старуху в земскую больницу.', 'Нужно ему проехать верст тридцать, а между тем дорога ужасная, с которой не справиться казенному почтарю, а не то что такому лежебоке, как токарь Григорий.', 'Прямо навстречу бьет резкий, холодный ветер.', 'В воздухе, куда ни взглянешь, кружатся целые облака снежинок, так что не разберешь, идет ли снег с неба, или с земли.', 'За снежным туманом не видно ни поля, ни телеграфных столбов, ни леса, а когда на Григория налетает особенно сильный порыв ветра, тогда не бывает видно даже дуги.', 'Дряхлая, слабосильная кобылка плетется еле-еле.', 'Вся энергия ее ушла на вытаскивание ног из глубокого снега и подергиванье головой.', 'Токарь торопится.', 'Он беспокойно прыгает на облучке и то и дело хлещет по лошадиной спине.']), ('«Гриша»', ['До сих пор Гриша знал один только четырехугольный мир, где в одном углу стоит его кровать, в другом — нянькин сундук, в третьем — стул, а в четвертом — горит лампадка.', 'Если взглянуть под кровать, то увидишь куклу с отломанной рукой и барабан, а за нянькиным сундуком очень много разных вещей: катушки от ниток, бумажки, коробка без крышки и сломанный паяц.', 'В этом мире, кроме няни и Гриши, часто бывают мама и кошка.', 'Мама похожа на куклу, а кошка на папину шубу, только у шубы нет глаз и хвоста.', 'Из мира, который называется детской, дверь ведет в пространство, где обедают и пьют чай.', 'Тут стоит Гришин стул на высоких ножках и висят часы, существующие для того только, чтобы махать маятником и звонить.']), ('«Дама с собачкой»', ['Ему не было еще сорока, но у него была уже дочь двенадцати лет и два сына-гимназиста.', 'Его женили рано, когда он был еще студентом второго курса, и теперь жена казалась в полтора раза старше его.', 'Это была женщина высокая, с темными бровями, прямая, важная, солидная и, как она сама себя называла, мыслящая.', 'Она много читала, не писала в письмах ъ, называла мужа не Дмитрием, а Димитрием, а он втайне считал ее недалекой, узкой, неизящной, боялся ее и не любил бывать дома.', 'Изменять ей он начал уже давно, изменял часто и, вероятно, поэтому о женщинах отзывался почти всегда дурно, и когда в его присутствии говорили о них, то он называл их так: — Низшая раса!']), ('«Дачники»', ['По дачной платформе взад и вперед прогуливалась парочка недавно поженившихся супругов.', 'Он держал ее за талию, а она жалась к нему, и оба были счастливы.', 'Из-за облачных обрывков глядела на них луна и хмурилась: вероятно, ей было завидно и досадно на свое скучное, никому не нужное девство.', 'Неподвижный воздух был густо насыщен запахом сирени и черемухи.', 'Где-то, по ту сторону рельсов, кричал коростель... — Как хорошо, Саша, как хорошо!', '— говорила жена.', '— Право, можно подумать, что всё это снится.', 'Ты посмотри, как уютно и ласково глядит этот лесок!', 'Как милы эти солидные, молчаливые телеграфные столбы!', 'Они, Саша, оживляют ландшафт и говорят, что там, где-то, есть люди... цивилизация... А разве тебе не нравится, когда до твоего слуха ветер слабо доносит шум идущего поезда?']), ('«Два скандала»', ['Она рада бы петь хорошо, не фальшивить... Она и умела не фальшивить, была мастером своего дела.', 'Но разве виновата она была, что ее глаза не повиновались ей?', 'Они, эти красивые, но недобросовестные глаза, которые она будет проклинать до самой смерти, они, вместо того чтобы глядеть в ноты и следить за движениями его палочки, смотрели в волосы и в глаза дирижера... Ее глазам нравились всклокоченные волосы и дирижерские глаза, из которых сыпались на нее искры и в которые страшно смотреть.', 'Бедная девочка без памяти любила лицо, по которому бегали тучи и молнии.', 'Разве виновата она была, что ее маленький ум, вместо того чтобы утонуть в репетиции, думал о посторонних вещах, которые мешают дело делать, жить, быть покойной...']), ('«Детвора»', ['Андрей бледнеет, кривит рот и хлоп Алешу по голове!', 'Алеша злобно таращит глаза, вскакивает, становится одним коленом на стол и, в свою очередь, — хлоп Андрея по щеке!', 'Оба дают друг другу еще по одной пощечине и ревут.', 'Соня, не выносящая таких ужасов, тоже начинает плакать, и столовая оглашается разноголосым ревом.', 'Но не думайте, что игра от этого кончилась.', 'Не проходит и пяти минут, как дети опять хохочут и мирно беседуют.', 'Лица заплаканы, но это не мешает им улыбаться.', 'Алеша даже счастлив: недоразумение было!', 'В столовую входит Вася, ученик V класса.', 'Вид у него заспанный, разочарованный.', '«Это возмутительно!', '— думает он, глядя, как Гриша ощупывает карман, в котором звякают копейки.', '— Разве можно давать детям деньги?', 'И разве можно позволять им играть в азартные игры?', 'Хороша педагогия, нечего сказать.', 'Возмутительно!»']), ('«Добрый знакомый»', ['По зеркальному льду скользят мужские ботфорты и женские ботинки с меховой опушкой.', 'Скользящих ног так много, что, будь они в Китае, для них не хватило бы бамбуковых палок.', 'Солнцу светит особенно ярко, воздух особенно прозрачен, щечки горят ярче обыкновенного, глазки обещают больше, чем следует... Живи и наслаждайся, человек, одним словом!', 'Но... «Дудки», — говорит судьба в лице моего... доброго знакомого.', 'Я вдали от катка сижу на скамье под голым деревом и беседую с «ней».', 'Я готов ее скушать вместе с ее шляпкой, шубкой и ножками, на которых блестят коньки — так хороша!', 'Страдаю и в то же время наслаждаюсь!', 'О, любовь!', 'Но... дудки...']), ('«Дом с мезонином»', ['Это было 6—7 лет тому назад, когда я жил в одном из уездов Т-ой губернии, в имении помещика Белокурова, молодого человека, который вставал очень рано, ходил в поддевке, по вечерам пил пиво и всё жаловался мне, что он нигде и ни в ком не встречает сочувствия.', 'Он жил в саду во флигеле, а я в старом барском доме, в громадной зале с колоннами, где не было никакой мебели, кроме широкого дивана, на котором я спал, да еще стола, на котором я раскладывал пасьянс.', 'Тут всегда, даже в тихую погоду, что-то гудело в старых амосовских печах, а во время грозы весь дом дрожал и, казалось, трескался на части, и было немножко страшно, особенно ночью, когда все десять больших окон вдруг освещались молнией.']), ('«Дома»', ['И подобные мысли, легкие и расплывчатые, какие приходят только в утомленный, отдыхающий мозг, стали бродить в голове Евгения Петровича; являются они неизвестно откуда и зачем, недолго остаются в голове и, кажется, ползают по поверхности мозга, не заходя далеко вглубь.', 'Для людей, обязанных по целым часам и даже дням думать казенно, в одном направлении, такие вольные, домашние мысли составляют своего рода комфорт, приятное удобство.', 'Был девятый час вечера.', 'Наверху, за потолком, во втором этаже кто-то ходил из угла в угол, а еще выше, на третьем этаже, четыре руки играли гаммы.', 'Шаганье человека, который, судя по нервной походке, о чем-то мучительно думал или же страдал зубною болью, и монотонные гаммы придавали тишине вечера что-то дремотное, располагающее к ленивым думам.', 'Через две комнаты в детской разговаривали гувернантка и Сережа.']), ('«Дорогая собака»', ['Дубов привлек к себе Милку и поцеловал ее между ушей.', 'На глазах у него выступили слезы.', '— Никому тебя не отдам... красавица моя... разбойник этакий.', 'Ведь ты любишь меня, Милка?', 'Любишь?..', 'Ну, пошла вон!', '— крикнул вдруг поручик.', '— Грязными лапами прямо на мундир лезешь!', 'Да, Кнапс, полтораста рублей дал, за щенка!', 'Стало быть, было за что!', 'Одно только жаль: охотиться мне некогда!', 'Гибнет без дела собака, талант свой зарывает... Потому-то и продаю.', 'Купите, Кнапс!', 'Всю жизнь будете благодарны!', 'Ну, если у вас денег мало, то извольте, я уступлю вам половину... Берите за пятьдесят!', 'Грабьте!', '— Нет, голубчик... — вздохнул Кнапс.', '— Будь ваша Милка мужеского пола, то, может быть, я и купил бы, а то...']), ('«Дорогие уроки»', ['— Извините, Алиса Осиповна, но я должен вам сказать... поставлен в тяжелую необходимость... Взглянув на конверт, француженка догадалась, в чем дело, и в первый раз за всё время уроков ее лицо дрогнуло и холодное, деловое выражение исчезло.', 'Она слегка зарумянилась и, опустив глаза, стала нервно перебирать пальцами свою тонкую золотую цепочку.', 'И Воротов, глядя на ее смущение, понял, как для нее дорог был рубль и как ей тяжело было бы лишиться этого заработка.', '— Я должен вам сказать... — пробормотал он, смущаясь еще больше, и в груди у него что-то екнуло; он торопливо сунул конверт в карман и продолжал: — Извините, я... я оставлю вас на десять минут...']), ('«Дочь Альбиона»', ['Отцов постоял, подумал и пошел к реке искать Грябова.', 'Нашел он его версты за две от дома, подойдя к реке.', 'Поглядев вниз с крутого берега и увидев Грябова, Отцов прыснул... Грябов, большой, толстый человек с очень большой головой, сидел на песочке, поджав под себя по-турецки ноги, и удил.', 'Шляпа у него была на затылке, галстук сполз набок.', 'Возле него стояла высокая, тонкая англичанка с выпуклыми рачьими глазами и большим птичьим носом, похожим скорей на крючок, чем на нос.', 'Одета она была в белое кисейное платье, сквозь которое сильно просвечивали тощие, желтые плечи.', 'На золотом поясе висели золотые часики.', 'Она тоже удила.', 'Вокруг обоих царила гробовая тишина.', 'Оба были неподвижны, как река, на которой плавали их поплавки.']), ('«Душечка»', ['Он всплеснул руками и продолжал, обращаясь к Оленьке: — Вот вам, Ольга Семеновна, наша жизнь.', 'Хоть плачь!', 'Работаешь, стараешься, мучишься, ночей не спишь, всё думаешь, как бы лучше, — и что же?', 'С одной стороны, публика, невежественная, дикая.', 'Даю ей самую лучшую оперетку, феерию, великолепных куплетистов, по разве ей это нужно?', 'Разве она в этом понимает что-нибудь?', 'Ей нужен балаган!', 'Ей подавай пошлость!', 'С другой стороны, взгляните на погоду.', 'Почти каждый вечер дождь.', 'Как зарядило с десятого мая, так потом весь май и июнь, просто ужас!', 'Публика не ходит, но ведь я за аренду плачу?', 'Артистам плачу?', 'На другой день под вечер опять надвигались тучи, и Кукин говорил с истерическим хохотом: — Ну что ж?', 'И пускай!', 'Пускай хоть весь сад зальет, хоть меня самого!', 'Чтоб мне не было счастья ни на этом, ни на том свете!', 'Пускай артисты подают на меня в суд!', 'Что суд?', 'Хоть на каторгу в Сибирь!', 'Хоть на эшафот!', 'Ха-ха-ха!']), ('«Дуэль»', ['Приятели оделись и пошли в павильон.', 'Тут Самойленко был своим человеком, и для него имелась даже особая посуда.', 'Каждое утро ему подавали на подносе чашку кофе, высокий граненый стакан с водою и со льдом и рюмку коньяку; он сначала выпивал коньяк, потом горячий кофе, потом воду со льдом, и это, должно быть, было очень вкусно, потому что после питья глаза у него становились маслеными, он обеими руками разглаживал бакены и говорил, глядя на море: — Удивительно великолепный вид!', 'После долгой ночи, потраченной на невеселые, бесполезные мысли, которые мешали спать и, казалось, усиливали духоту и мрак ночи, Лаевский чувствовал себя разбитым и вялым.', 'От купанья и кофе ему не стало лучше.']), ('«Егерь»', ['Знойный и душный полдень.', 'На небе ни облачка... Выжженная солнцем трава глядит уныло, безнадежно: хоть и будет дождь, но уж не зеленеть ей... Лес стоит молча, неподвижно, словно всматривается куда-то своими верхушками или ждет чего-то.', 'По краю сечи лениво, вразвалку, плетется высокий узкоплечий мужчина лет сорока, в красной рубахе, латаных господских штанах и в больших сапогах.', 'Плетется он по дороге.', 'Направо зеленеет сеча, налево, до самого горизонта, тянется золотистое море поспевшей ржи... Он красен и вспотел.', 'На его красивой белокурой голове ухарски сидит белый картузик с прямым, жокейским козырьком, очевидно подарок какого-нибудь расщедрившегося барича.', 'Через плечо перекинут ягдташ, в котором лежит скомканный петух-тетерев.', 'Мужчина держит в руках двустволку со взведенными курками и щурит глаза на своего старого, тощего пса, который бежит впереди и обнюхивает кустарник.', 'Кругом тихо, ни звука... Всё живое попряталось от зноя.']), ('«Жена»', ['Что касается моих домашних, то ждать от них помощи или поддержки я мог меньше всего.', 'От моей первой, отцовской, когда-то большой и шумной семьи уцелела одна только гувернантка m-lle Marie, или, как ее звали теперь, Марья Герасимовна, личность совершенно ничтожная.', 'Эта маленькая, аккуратная старушка лет семидесяти, одетая в светло-серое платье и чепец с белыми лентами, похожая на фарфоровую куклу, всегда сидела в гостиной и читала книгу.', 'Когда я проходил мимо нее, она, зная причину моего раздумья, всякий раз говорила: — Что же вы хотите, Паша?', 'Я и раньше говорила, что это так будет.', 'Вы по нашей прислуге можете судить.', 'Моя вторая семья, то есть жена Наталья Гавриловна, жила в нижнем этаже, в котором занимала все комнаты.']), ('«Живой товар»', ['— Ну да, что делать?', 'Решай вот, мудрая головка... Я люблю тебя, а любящий человек не подельчив.', 'Он более чем эгоист.', 'Я не в силах делиться с твоим мужем.', 'Я мысленно рву его на клочки, когда думаю, что и он любит тебя.', 'Во-вторых, ты любишь меня... Для любви необходимым условием является полная свобода... А ты разве свободна?', 'Тебя разве не терзает мысль, что над твоей душой вечно торчит этот человек?', 'Человек, которого ты не любишь, быть может, что очень естественно, ненавидишь... Это во-вторых... В-третьих же... Что же в-третьих?', 'А вот что... Мы обманываем его, а это... нечестно.', 'Прежде всего, Лиза, правда.', 'Прочь ложь!']), ('«Жизнь в вопросах и восклицаниях»', ['Кого бог дал, сына или дочь?', 'Крестить скоро?', 'Крупный мальчик!', 'Не урони, мамка!', 'Ах, ах!', 'Упадет!!', 'Зубки прорезались?', 'Это у него золотуха?', 'Возьмите у него кошку, а то она его оцарапает!', 'Потяни дядю за ус!', 'Так!', 'Не плачь!', 'Домовой идет!', 'Он уже и ходить умеет!', 'Унесите его отсюда — он невежлив!', 'Что он вам наделал?!', 'Бедный сюртук!', 'Ну, ничего, мы высушим!', 'Чернило опрокинул!', 'Спи, пузырь!', 'Он уже говорит!', 'Ах, какая радость!', 'А ну-ка, скажи что-нибудь!', 'Чуть извозчики не задавили!!', 'Прогнать няньку!', 'Не стой на сквозном ветре!', 'Постыдитесь, можно ли бить такого маленького?', 'Не плачь!', 'Дайте ему пряник!', 'Иди-ка сюда, я тебя высеку!', 'Где это ты себе нос разбил?', 'Не беспокой мамашу!', 'Ты не маленький!', 'Не подходи к столу, тебе после!', 'Читай!', 'Не знаешь?', 'Пошел в угол!', 'Единица!', 'Не клади в карман гвоздей!', 'Почему ты мамаши не слушаешься?', 'Ешь как следует!', 'Не ковыряй в носу!', 'Это ты ударил Митю?']), ('Жизнь прекрасна!', ['Когда у тебя в кармане загораются спички, то радуйся и благодари небо, что у тебя в кармане не пороховой погреб.', 'Когда к тебе на дачу приезжают бедные родственники, то не бледней, а торжествуя восклицай: «Хорошо, что это не городовые!» Когда в твой палец попадает заноза, радуйся: «Хорошо, что не в глаз!» Если твоя жена или свояченица играет гаммы, то не выходи из себя, а не находи себе места от радости, что ты слушаешь игру, а не вой шакалов или кошачий концерт.', 'Радуйся, что ты не лошадь конножелезки, не коховская «запятая», не трихина, не свинья, не осел, не медведь, которого водят цыгане, не клоп...']), ('«За двумя зайцами погонишься, ни одного не поймаешь»', ['Вчера, проходя мимо беседки, он слышал, как молодая жена его, майорша Каролина Карловна, более чем милостиво беседовала со своим приезжим кузеном, называла своего супруга, майора Щелколобова, бараном и с женским легкомыслием доказывала, что она своего мужа не любила, не любит и любить не будет за его, Щелколобова, тупоумие, мужицкие манеры и наклонность к умопомешательству и хроническому пьянству.', 'Такое отношение жены поразило, возмутило и привело в сильнейшее негодование майора.', 'Он не спал целую ночь и целое утро.', 'В голове у него кипела непривычная работа, лицо горело и было краснее вареного рака; кулаки судорожно сжимались, а в груди происходила такая возня и стукотня, какой майор и под Карсом не видал и не слыхал.', 'Выглянув из-под одеяла на свет божий и выругавшись, он спрыгнул с кровати и, потрясая кулаками, зашагал по комнате.']), ('«За яблочки»', ['Карпушка начал рвать крапиву.', 'Парень очень хорошо знал, для чего это готовилась крапива.', 'Трифон Семенович, подобно ему подобным, красиво самоуправничает.', 'Вора он или запирает на сутки в погреб, или сечет крапивой, или же отпускает на свою волю, предварительно только раздев его донага... Это для вас ново?', 'Но есть люди и места, для которых это обыденно и старо, как телега.', 'Григорий косо посмотрел на крапиву, помялся, покашлял и начал не рассказывать сказку, а молоть сказку.', 'Кряхтя, потея, кашляя, поминутно сморкаясь, начал он повествовать о том, как во время оно богатыри русские кощеев колотили да на красавицах женились.', 'Трифон Семенович стоял, слушал и не спускал глаз с повествователя.']), ('«Забыл!!»', ['Играл когда-то на скрипке, на одной струне, да и то так... сдуру... Меня не учили... Брат мой Назар играет... Того учили... Француз Рокат, может быть, знаете, Венедикт Францыч учил... Такой потешный французишка... Мы его Буонапартом дразнили... Сердился... «Я, говорит, не Буонапарт... Я республик Франце»... И рожа у него, по правде сказать, была республиканская... Совсем собачья рожа... Меня покойный мой родитель ничему не учил... Деда, говаривал, твоего Иваном звали, и ты Иван, а потому ты должен быть подобен деду своему во всех своих поступках: на военную, прохвост!', 'Пороху!!', 'Нежностей, брат... брат... Я, брат... Я, брат, нежностей тебе не дозволяю!', 'Дед, в некотором роде, кониной питался, и ты оной питайся!', 'Седло под головы себе клади вместо подушки!..', 'Будет мне теперь дома!', 'Заедят!', 'Без нот и приезжать не велено... Прощайте-с, в таком случае!', 'Извините за беспокойство!..', 'Сколько эта рояля стоит?']), ('«Загадочная натура»', ['— Но вот старик умер... Мне он оставил кое-что, я свободна, как птица.', 'Теперь-то и жить мне счастливо... Не правда ли, Вольдемар?', 'Счастье стучится ко мне в окно.', 'Стоит только впустить его, но... нет!', 'Вольдемар, слушайте, заклинаю вас!', 'Теперь-то и отдаться любимому человеку, сделаться его подругой, помощницей, носительницей его идеалов, быть счастливой... отдохнуть... Но как всё пошло, гадко и глупо на этом свете!', 'Как всё подло, Вольдемар!', 'Я несчастна, несчастна, несчастна!', 'На моем пути опять стоит препятствие!', 'Опять я чувствую, что счастье мое далеко, далеко!', 'Ах, сколько мук, если б вы знали!', 'Сколько мук!', '— Но что же?', 'Что стало на вашем пути?', 'Умоляю вас, говорите!', 'Что же?']), ('«Зиночка»', ['Компания охотников ночевала в мужицкой избе на свежем сене.', 'В окна глядела луна, на улице грустно пиликала гармоника, сено издавало приторный, слегка возбуждающий запах.', 'Охотники говорили о собаках, о женщинах, о первой любви, о бекасах.', 'После того как были перебраны косточки всех знакомых барынь и была рассказана сотня анекдотов, самый толстый из охотников, похожий в потемках на копну сена и говоривший густым штаб-офицерским басом, громко зевнул и сказал: — Не велика штука быть любимым: барыни на то и созданы, чтоб любить нашего брата.', 'А вот, господа, был ли кто-нибудь из вас ненавидим, ненавидим страстно, бешено?', 'Не наблюдал ли кто-нибудь из вас восторгов ненависти?', 'А?']), ('«Злой мальчик»', ['Иван Иваныч Лапкин, молодой человек приятной наружности, и Анна Семеновна Замблицкая, молодая девушка со вздернутым носиком, спустились вниз по крутому берегу и уселись на скамеечке.', 'Скамеечка стояла у самой воды, между густыми кустами молодого ивняка.', 'Чудное местечко!', 'Сели вы тут, и вы скрыты от мира — видят вас одни только рыбы да пауки-плауны, молнией бегающие по воде.', 'Молодые люди были вооружены удочками, сачками, банками с червями и прочими рыболовными принадлежностями.', 'Усевшись, они тотчас же принялись за рыбную ловлю.', '— Я рад, что мы наконец одни, — начал Лапкин, оглядываясь.', '— Я должен сказать вам многое, Анна Семеновна... Очень многое... Когда я увидел вас в первый раз... У вас клюет... Я понял тогда, для чего я живу, понял, где мой кумир, которому я должен посвятить свою честную, трудовую жизнь...']), ('«Идиллия — увы и ах!»', ['И слезы навертывались на глазах Гриши, когда он говорил о дядюшке.', 'К чести его сказать, он не стыдился этих хороших слез и плакал публично!', 'Я внял его просьбам и неделю тому назад зашел к капитану.', 'Когда я вошел в переднюю и заглянул в залу, я увидел умилительную картину.', 'В большом кресле среди залы сидел старенький, худенький капитан и кушал чай.', 'Перед ним на одном колене стоял Гриша и с умилением мешал ложечкой его чай.', 'Вокруг коричневой шеи старичка обвивалась хорошенькая ручка Гришиной невесты... бедный племянник и невеста спорили о том, кто из них скорей поцелует дядюшку, и не жалели поцелуев для старичка.']), ('«Ионыч»', ['С полверсты он прошел полем.', 'Кладбище обозначалось вдали темной полосой, как лес или большой сад.', 'Показалась ограда из белого камня, ворота... При лунном свете на воротах можно было прочесть: «Грядет час в онь же...» Старцев вошел в калитку, и первое, что он увидел, это белые кресты и памятники по обе стороны широкой аллеи и черные тени от них и от тополей; и кругом далеко было видно белое и черное, и сонные деревья склоняли свои ветви над белым.', 'Казалось, что здесь было светлей, чем в поле; листья кленов, похожие на лапы, резко выделялись на желтом песке аллей и на плитах, и надписи на памятниках были ясны.']), ('«История одного торгового предприятия»', ['Нанявши помещение под магазин, он съездил в Москву и привез оттуда много старых и новейших авторов и много учебников, и расставил все это добро по полкам.', 'В первые три недели покупатели совсем не приходили.', 'Андрей Андреевич сидел за прилавком, читал Михайловского и старался честно мыслить.', 'Когда же ему невзначай приходило в голову, например, что недурно бы теперь покушать леща с кашей, то он тотчас же ловил себя на этих мыслях: «Ах, как пошло!» Каждый день утром в магазин опрометью вбегала озябшая девка в платке и в кожаных калошах на босую ногу и говорила: — Дай на две копейки уксусу!', 'И Андрей Андреевич с презрением отвечал ей: — Дверью ошиблись, сударыня!']), ('«Каштанка»', ['Каштанка бегала взад и вперед и не находила хозяина, а между тем становилось темно.', 'По обе стороны улицы зажглись фонари и в окнах домов показались огни.', 'Шел крупный, пушистый снег и красил в белое мостовую, лошадиные спины, шапки извозчиков, и чем больше темнел воздух, тем белее становились предметы.', 'Мимо Каштанки, заслоняя ей поле зрения и толкая ее ногами, безостановочно взад и вперед проходили незнакомые заказчики.', '(Всё человечество Каштанка делила на две очень неравные части: на хозяев и на заказчиков; между теми и другими была существенная разница: первые имели право бить ее, а вторых она сама имела право хватать за икры.)', 'Заказчики куда-то спешили и не обращали на нее никакого внимания.']), ('«Корреспондент»', ['Музыкантов было восемь человек.', 'Главе их, Гурию Максимову, было заявлено, что если музыка не будет играть неумолкаемо, то музыканты не увидят ни одной рюмки водки и благодарность за труд получат с великой натяжкой.', 'Танцы начались ровно в восемь часов вечера.', 'В час ночи барышни обиделись на кавалеров; полупьяные кавалеры обиделись на барышень, и танцы расстроились.', 'Гости разделились на труппы.', 'Старички заняли гостиную, в которой стоял стол с сорока четырьмя бутылками и со столькими же тарелками; барышни забились в уголок, зашептали о безобразиях кавалеров и стали решать вопрос: как это так выходит, что невеста с первого же раза начинает говорить на жениха «ты»?', 'Кавалеры заняли другой угол и заговорили все разом, каждый про свое.', 'Гурий, первая и плохая скрипка и дирижер, заиграл со своими семью черняевский марш...']), ('«Кот»', ['Поглядев на небо, Алеша увидел одну только звездочку, да и та мерцала точно в тумане, еле-еле... В липе заворчали воробьи, испуганные шумом открывающегося окна.', 'Алеша поглядел вниз на землю и увидел штук десять котов.', 'Вытянув хвосты, шипя и нежно ступая по травке, они дромадерами ходили вокруг хорошенькой кошечки, сидевшей на опрокинутой вверх дном лохани, и пели.', 'Трудно было решить, чего в них было больше: любви ли к кошечке, или собственного достоинства?', 'За любовью ли они пришли, или только за тем, чтобы достоинство свое показать?', 'В отношениях друг к другу сквозила самая утонченная ненависть... По ту сторону палисадника терлась о решетку свинья с поросятами и просилась в садик.']), ('«Крыжовник»', ['Еще с раннего утра всё небо обложили дождевые тучи; было тихо, не жарко и скучно, как бывает в серые пасмурные дни, когда над полем давно уже нависли тучи, ждешь дождя, а его нет.', 'Ветеринарный врач Иван Иваныч и учитель гимназии Буркин уже утомились идти, и поле представлялось им бесконечным.', 'Далеко впереди еле были видны ветряные мельницы села Мироносицкого, справа тянулся и потом исчезал далеко за селом ряд холмов, и оба они знали, что это берег реки, там луга, зеленые ивы, усадьбы, и если стать на один из холмов, то оттуда видно такое же громадное поле, телеграф и поезд, который издали похож на ползущую гусеницу, а в ясную погоду оттуда бывает виден даже город.', 'Теперь, в тихую погоду, когда вся природа казалась кроткой и задумчивой, Иван Иваныч и Буркин были проникнуты любовью к этому полю и оба думали о том, как велика, как прекрасна эта страна.']), ('«Кухарка женится»', ['«Кухарка женится... — думал он.', '— Странно.', 'Не понимаю, зачем это жениться?', 'Мамаша женилась на папаше, кузина Верочка — на Павле Андреиче.', 'Но на папе и Павле Андреиче, так и быть уж, можно жениться: у них есть золотые цепочки, хорошие костюмы, у них всегда сапоги вычищенные; но жениться на этом страшном извозчике с красным носом, в валенках... фи!', 'И почему это няньке хочется, чтоб бедная Пелагея женилась?» Когда из кухни ушел гость, Пелагея явилась в комнаты и занялась уборкой.', 'Волнение еще не оставило ее.', 'Лицо ее было красно и словно испуганно.', 'Она едва касалась веником пола и по пяти раз мела каждый угол.', 'Долго она не выходила из той комнаты, где сидела мамаша.', 'Ее, очевидно, тяготило одиночество и ей хотелось высказаться, поделиться с кем-нибудь впечатлениями, излить душу.']), ('«Лошадиная фамилия»', ['У отставного генерал-майора Булдеева разболелись зубы.', 'Он полоскал рот водкой, коньяком, прикладывал к больному зубу табачную копоть, опий, скипидар, керосин, мазал щеку йодом, в ушах у него была вата, смоченная в спирту, но всё это или не помогало, или вызывало тошноту.', 'Приезжал доктор.', 'Он поковырял в зубе, прописал хину, но и это не помогло.', 'На предложение вырвать больной зуб генерал ответил отказом.', 'Все домашние — жена, дети, прислуга, даже поваренок Петька предлагали каждый свое средство.', 'Между прочим и приказчик Булдеева Иван Евсеич пришел к нему и посоветовал полечиться заговором.', '— Тут, в нашем уезде, ваше превосходительство, — сказал он, — лет десять назад служил акцизный Яков Васильич.', 'Заговаривал зубы — первый сорт.', 'Бывало, отвернется к окошку, пошепчет, поплюет — и как рукой!', 'Сила ему такая дадена...']), ('«Мальчики»', ['Вся семья Королевых, с часу на час поджидавшая своего Володю, бросилась к окнам.', 'У подъезда стояли широкие розвальни, и от тройки белых лошадей шел густой туман.', 'Сани были пусты, потому что Володя уже стоял в сенях и красными, озябшими пальцами развязывал башлык.', 'Его гимназическое пальто, фуражка, калоши и волосы на висках были покрыты инеем, и весь он от головы до ног издавал такой вкусный морозный запах, что, глядя на него, хотелось озябнуть и сказать: «Бррр!» Мать и тетка бросились обнимать и целовать его, Наталья повалилась к его ногам и начала стаскивать с него валенки, сестры подняли визг, двери скрипели, хлопали, а отец Володи в одной жилетке и с ножницами в руках вбежал в переднюю и закричал испуганно:']), ('«Маска»', ['Мужчина поднялся, вытянулся во весь рост и сорвал с себя маску.', 'Открыв свое пьяное лицо и поглядев на всех, любуясь произведенным эффектом, он упал в кресло и радостно захохотал.', 'А впечатление, действительно, произвел он необыкновенное.', 'Все интеллигенты растерянно переглянулись и побледнели, некоторые почесали затылки.', 'Евстрат Спиридоныч крякнул, как человек, сделавший нечаянно большую глупость.', 'В буяне все узнали местного миллионера, фабриканта, потомственного почетного гражданина Пятигорова, известного своими скандалами, благотворительностью и, как не раз говорилось в местном вестнике, — любовью к просвещению.', '— Что ж, уйдете или нет?', '— спросил Пятигоров после минутного молчания.', 'Интеллигенты молча, не говоря ни слова, вышли на цыпочках из читальни, и Пятигоров запер за ними двери.']), ('«Мелюзга»', ['«Милостивый государь, отец и благодетель!» — сочинял начерно чиновник Невыразимов поздравительное письмо.', '«Желаю как сей Светлый день, так и многие предбудущие провести в добром здравии и благополучии.', 'А также и семейству жел...» Лампа, в которой керосин был уже на исходе, коптила и воняла гарью.', 'По столу, около пишущей руки Невыразимова, бегал встревоженно заблудившийся таракан.', 'Через две комнаты от дежурной швейцар Парамон чистил уже в третий раз свои парадные сапоги, и с такой энергией, что его плевки и шум ваксельной щетки были слышны во всех комнатах.', '«Что бы еще такое ему, подлецу, написать?» — задумался Невыразимов, поднимая глаза на закопченный потолок.']), ('«Месть»', ['Лев Саввич Турманов, дюжинный обыватель, имеющий капиталец, молодую жену и солидную плешь, как-то играл на именинах у приятеля в винт.', 'После одного хорошего минуса, когда его в пот ударило, он вдруг вспомнил, что давно не пил водки.', 'Поднявшись, он на цыпочках, солидно покачиваясь, пробрался между столов, прошел через гостиную, где танцевала молодежь (тут он снисходительно улыбнулся и отечески похлопал по плечу молодого жидкого аптекаря), затем юркнул в маленькую дверь, которая вела в буфетную.', 'Тут, на круглом столике, стояли бутылки, графины с водкой... Около них, среди другой закуски, зеленея луком и петрушкой, лежала на тарелке наполовину уже съеденная селедка.', 'Лев Саввич налил себе рюмку, пошевелил в воздухе пальцами, как бы собираясь говорить речь, выпил и сделал страдальческое лицо, потом ткнул вилкой в селедку и... Но тут за стеной послышались голоса.']), ('«Моя жизнь»', ['Управляющий сказал мне: «Держу вас только из уважения к вашему почтенному батюшке, а то бы вы у меня давно полетели».', 'Я ему ответил: «Вы слишком льстите мне, ваше превосходительство, полагая, что я умею летать».', 'И потом я слышал, как он сказал: «Уберите этого господина, он портит мне нервы».', 'Дня через два меня уволили.', 'Итак, за все время, пока я считаюсь взрослым, к великому огорчению моего отца, городского архитектора, я переменил девять должностей.', 'Я служил по различным ведомствам, но все эти девять должностей были похожи одна на другую, как капли воды: я должен был сидеть, писать, выслушивать глупые или грубые замечания и ждать, когда меня уволят.']), ('«Мужики»', ['Приехал он в свое Жуково под вечер.', 'В воспоминаниях детства родное гнездо представлялось ему светлым, уютным, удобным, теперь же, войдя в избу, он даже испугался: так было темно, тесно и нечисто.', 'Приехавшие с ним жена Ольга и дочь Саша с недоумением поглядывали на большую неопрятную печь, занимавшую чуть ли не пол-избы, темную от копоти и мух.', 'Сколько мух!', 'Печь покосилась, бревна в стенах лежали криво, и казалось, что изба сию минуту развалится.', 'В переднем углу, возле икон, были наклеены бутылочные ярлыки и обрывки газетной бумаги — это вместо картин.', 'Бедность, бедность!', 'Из взрослых никого не было дома, все жали.', 'На печи сидела девочка лет восьми, белоголовая, немытая, равнодушная; она даже не взглянула на вошедших.', 'Внизу терлась о рогач белая кошка.']), ('«На волчьей садке»', ['Первый час.', 'Позади галереи толпятся кареты, роскошные сани и извозчики.', 'Шум, гвалт... Экипажей так много, что приходится толпиться... В галереях конского бега, в енотах, бобрах, лисицах и барашках, заседают жеребятники, кобелятники, борзятники, перепелятники и прочие лятники, мерзнут и сгорают от нетерпения.', 'Тут же заседают, разумеется, и дамы... Без дам нигде дело не обходится.', 'Хорошеньких, сверх обыкновения, почему-то очень много... Дам столько же, сколько и мужчин... Они тоже сгорают от нетерпения.', 'На верхних скамьях мелькают гимназические фуражки.', 'И гимназисты пришли поглазеть, и они тоже сгорают от нетерпения и постукивают калошами.', 'Любители, ценители и критиканы, приперевшие на Ходынку пехтурой и не имеющие рубля, толпятся у заборов, по колена в снегу, вытягивают шеи и тоже сгорают от нетерпения.']), ('«На мельнице»', ['Мельник Алексей Бирюков, здоровенный, коренастый мужчина средних лет, фигурой и лицом похожий на тех топорных, толстокожих и тяжело ступающих матросов, которые снятся детям после чтения Жюля Верна, сидел у порога своей хижины и лениво сосал потухшую трубку.', 'На этот раз он был в серых штанах из грубого солдатского сукна, в больших тяжелых сапогах, но без сюртука и без шапки, хотя на дворе стояла настоящая осень, сырая и холодная.', 'Сквозь расстегнутую жилетку свободно проникала сырая мгла, но большое, черствое, как мозоль, тело мельника, по-видимому, не ощущало холода.', 'Красное, мясистое лицо его по обыкновению было апатично и дрябло, точно спросонок, маленькие, заплывшие глазки угрюмо исподлобья глядели по сторонам то на плотину, то на два сарая с навесами, то на старые, неуклюжие ветлы.']), ('«На охоте»', ['Дядя встретил меня с распростертыми объятиями и, как это водится даже у самых гостеприимных охотников, не дав мне оправиться после долгой дороги и отдохнуть, повел меня на псарню показывать мне своих лошадей и собак.', 'Собаки, по моему мнению, бывают большие, маленькие и средние, белые, черные и серые, злые и смирные; дядя же различал между ними крапчатых, темно-багряных, сохастовых, лещеватых, черно-пегих, черных в подпалинах, брудастых — совсем собачий язык, и мне кажется, что если бы собаки умели говорить, то говорили бы именно на таком языке.', 'Дядя показывал, целовал собак в морды и всё требовал, чтобы я щупал собачьи морды, трогал лапы.']), ('«На пути»', ['В комнате, которую сам содержатель трактира, казак Семен Чистоплюй, называет «проезжающей», то есть назначенной исключительно для проезжих, за большим некрашеным столом сидел высокий широкоплечий мужчина лет сорока.', 'Облокотившись о стол и подперев голову кулаком, он спал.', 'Огарок сальной свечи, воткнутый в баночку из-под помады, освещал его русую бороду, толстый широкий нос, загорелые щеки, густые черные брови, нависшие над закрытыми глазами... И нос, и щеки, и брови, все черты, каждая в отдельности, были грубы и тяжелы, как мебель и печка в «проезжающей», но в общем они давали нечто гармоническое и даже красивое.', 'Такова уж, как говорится, планида русского лица: чем крупнее и резче его черты, тем кажется оно мягче и добродушнее.', 'Одет был мужчина в господский пиджак, поношенный, но обшитый новой широкой тесьмой, в плюшевую жилетку и широкие черные панталоны, засунутые в большие сапоги.']), ('«Налим»', ['Летнее утро.', 'В воздухе тишина; только поскрипывает на берегу кузнечик да где-то робко мурлыкает орличка.', 'На небе неподвижно стоят перистые облака, похожие на рассыпанный снег... Около строящейся купальни, под зелеными ветвями ивняка, барахтается в воде плотник Герасим, высокий, тощий мужик с рыжей курчавой головой и с лицом, поросшим волосами.', 'Он пыхтит, отдувается и, сильно мигая глазами, старается достать что-то из-под корней ивняка.', 'Лицо его покрыто потом.', 'На сажень от Герасима, по горло в воде, стоит плотник Любим, молодой горбатый мужик с треугольным лицом и с узкими, китайскими глазками.', 'Как Герасим, так и Любим, оба в рубахах и портах.', 'Оба посипели от холода, потому что уж больше часа сидят в воде...']), ('«Нарвался»', ['«Спать хочется!', '— думал я, сидя в банке.', '— Приду домой и завалюсь спать».', '— Какое блаженство!', '— шептал я, наскоро пообедав и стоя перед своей кроватью.', '— Хорошо жить на этом свете!', 'Важно!', 'Бесконечно улыбаясь, потягиваясь и нежась на кровати, как кот на солнце, я закрыл глаза и принялся засыпать.', 'В закрытых глазах забегали мурашки; в голове завертелся туман, замахали крылья, полетели к небу из головы какие-то меха... с неба поползла в голову вата... Всё такое большое, мягкое, пушистое, туманное.', 'В тумане забегали маленькие человечки.', 'Они побегали, покрутились и скрылись за туманом... Когда исчез последний человечек и дело Морфея было уже в шляпе, я вздрогнул.']), ('«Не в духе»', ['Но и мука не развеселила Прачкина.', 'Чем более он утешал себя, тем чувствительнее становилась для него потеря.', 'Так было жалко восьми рублей, так жалко, точно он в самом деле проиграл восемь тысяч.', 'Когда Ваня кончил урок и умолк, Прачкин стал у окна и, тоскуя, вперил свой печальный взор в снежные сугробы... Но вид сугробов только растеребил его сердечную рану.', 'Он напомнил ему о вчерашней поездке к воинскому начальнику.', 'Заиграла желчь, подкатило под душу... Потребность излить на чем-нибудь свое горе достигла степеней, не терпящих отлагательства.', 'Он не вынес... — Ваня!', '— крикнул он.', '— Иди, я тебя высеку за то, что ты вчера стекло разбил!']), ('«Невеста»', ['Вот кто-то вышел из дома и остановился на крыльце: это Александр Тимофеич, или, попросту, Саша, гость, приехавший из Москвы дней десять назад.', 'Когда-то давно к бабушке хаживала за подаяньем ее дальняя родственница, Марья Петровна, обедневшая дворянка-вдова, маленькая, худенькая, больная.', 'У нее был сын Саша.', 'Почему-то про него говорили, что он прекрасный художник, и, когда у него умерла мать, бабушка, ради спасения души, отправила его в Москву в Комиссаровское училище; года через два перешел он в Училище живописи, пробыл здесь чуть ли не пятнадцать лет и кончил по архитектурному отделению, с грехом пополам, но архитектурой все-таки не занимался, а служил в одной из московских литографий.', 'Почти каждое лето приезжал он, обыкновенно очень больной, к бабушке, чтобы отдохнуть и поправиться.']), ('«Ненастье»', ['В темные окна стучали крупные дождевые капли.', 'Это был один из тех противных дачных дождей, которые обыкновенно, раз начавшись, тянутся долго, по неделям, пока озябнувший дачник, привыкнув, не погружается в совершенную апатию.', 'Было холодно, чувствовалась резкая, неприятная сырость.', 'Теща присяжного поверенного Квашина и его жена, Надежда Филипповна, одетые в ватерпруфы и шали, сидели в столовой за обеденным столом.', 'На лице старухи было написано, что она, слава богу, сыта, одета, здорова, выдала единственную дочку за хорошего человека и теперь со спокойного совестью может раскладывать пасьянс; дочь ее, небольшая полная блондинка лет двадцати, с кротким малокровным лицом, поставив локти на стол, читала книгу; судя по глазам, она не столько читала, сколько думала свои собственные мысли, которых не было в книге.', 'Обе молчали.', 'Слышался шум дождя, и из кухни доносились протяжные зевки кухарки.']), ('«Ненужная победа»', ['В июне венгерская степь дает себя знать.', 'Земля трескается, и дорога обращается в реку, в которой вместо воды волнуется серая пыль.', 'Ветер, если он и есть, горяч и сушит кожу.', 'В воздухе тишина от утра до вечера.', 'Тишина эта наводит на путника тоску.', 'Один только роскошные, всему свету известные, венгерские сады и виноградники не блекнут, не желтеют и не сохнут под жгучими лучами степного солнца.', 'Они, разбросанные рукою культурного человека по сторонам многочисленных рек и речек, от ранней весны до средины осени щеголяют своею зеленью, манят к себе прохожего и служат убежищем всего живого, бегущего от солнца.', 'В них царят тень, прохлада и чудный воздух.']), ('«Нищий»', ['Оборвыш пожал плечами, как бы недоумевая, и нерешительно пошел за кухаркой.', 'По его походке видно было, что согласился он идти колоть дрова не потому, что был голоден и хотел заработать, а просто из самолюбия и стыда, как пойманный на слове.', 'Заметно было также, что он сильно ослабел от водки, был нездоров и не чувствовал ни малейшего расположения к работе.', 'Скворцов поспешил в столовую.', 'Там из окон, выходивших на двор, виден был дровяной сарай и всё, что происходило на дворе.', 'Стоя у окна, Скворцов видел, как кухарка и оборвыш вышли черным ходом на двор и по грязному снегу направились к сараю.', 'Ольга, сердито оглядывая своего спутника и тыча в стороны локтями, отперла сарай и со злобой хлопнула дверью.']), ('«О женщины, женщины!»', ['— Вообще скверно, а сегодня в особенности.', 'Петров не хочет больше отпускать в кредит бумагу.', 'Секретарь запьянствовал... Но всё это пустяки, уладится как-нибудь... А вот где беда, Манечка... Сижу я сегодня в редакции и читаю корректуру своей передовой.', 'Вдруг, знаешь, отворяется дверь и входит князь Прочуханцев, давнишний мой друг и приятель, тот самый, что в любительских спектаклях всегда первых любовников играет и что актрисе Зрякиной за один поцелуй свою белую лошадь отдал.', '«Зачем, думаю, черти принесли?', 'Это недаром... Зрякиной, думаю, пришел рекламу делать»... Разговорились... То да се, пятое, десятое... Оказывается, что не за рекламой пришел.', 'Стихи свои принес для напечатания...']), ('«О любви»', ['На другой день к завтраку подавали очень вкусные пирожки, раков и бараньи котлеты; и пока ели, приходил наверх повар Никанор справиться, что гости желают к обеду.', 'Это был человек среднего роста, с пухлым лицом и маленькими глазами, бритый, и казалось, что усы у него были не бриты, а выщипаны.', 'Алехин рассказал, что красивая Пелагея была влюблена в этого повара.', 'Так как он был пьяница и буйного нрава, то она не хотела за него замуж, но соглашалась жить так.', 'Он же был очень набожен, и религиозные убеждения не позволяли ему жить так; он требовал, чтобы она шла за него, и иначе не хотел, и бранил ее, когда бывал пьян, и даже бил.', 'Когда он бывал пьян, она пряталась наверху и рыдала, и тогда Алехин и прислуга не уходили из дому, чтобы защитить ее в случае надобности.']), ('«Огни»', ['Ночь была августовская, звездная, но темная.', 'Оттого, что раньше я никогда в жизни не находился при такой исключительной обстановке, в какую попал случайно теперь, эта звездная ночь казалась мне глухой, неприветливой и темнее, чем она была на самом деле.', 'Я был на линии железной дороги, которая еще только строилась.', 'Высокая, наполовину готовая насыпь, кучи песку, глины и щебня, бараки, ямы, разбросанные кое-где тачки, плоские возвышения над землянками, в которых жили рабочие, — весь этот ералаш, выкрашенный потемками в один цвет, придавал земле какую-то странную, дикую физиономию, напоминавшую о временах хаоса.', 'Во всем, что лежало передо мной, было до того мало порядка, что среди безобразно изрытой, ни на что не похожей земли как-то странно было видеть силуэты людей и стройные телеграфные столбы; те и другие портили ансамбль картины и казались не от мира сего.', 'Было тихо, и только слышалось, как над нашими головами, где-то очень высоко, телеграф гудел свою скучную песню.']), ('«Он и она»', ['Коли хотите их видеть, приобретите право присутствовать на обедах, которые даются ей и которые иногда она сама дает перед отъездом из одной столицы в другую.', 'Приобрести это право легко только на первый взгляд, добраться же до обеденного стола могут только люди избранные... К последним относятся господа рецензенты, пролазы, выдающие себя за рецензентов, туземные певцы, дирижеры и капельмейстеры, любители и ценители с зализанными лысинами, попавшие в театральные завсегдатаи и блюдолизы благодаря злату, сребру и родству.', 'Обеды эти выходят не скучные, для человека наблюдающего интересные... Раза два стоит пообедать.', 'Известные (между обедающими много таких) едят и говорят.', 'Поза их вольная: шея на один бок, голова на другой, один локоть — на столе.', 'Старички даже ковыряют в зубах.']), ('«Орден»', ['Учитель военной прогимназии, коллежский регистратор Лев Пустяков, обитал рядом с другом своим, поручиком Леденцовым.', 'К последнему он и направил свои стопы в новогоднее утро.', '— Видишь ли, в чем дело, Гриша, — сказал он поручику после обычного поздравления с Новым годом.', '— Я не стал бы тебя беспокоить, если бы не крайняя надобность.', 'Одолжи мне, голубчик, на сегодняшний день твоего Станислава.', 'Сегодня, видишь ли, я обедаю у купца Спичкина.', 'А ты знаешь этого подлеца Спичкина: он страшно любит ордена и чуть ли не мерзавцами считает тех, у кого не болтается что-нибудь на шее или в петлице.', 'И к тому же у него две дочери... Настя, знаешь, и Зина... Говорю, как другу... Ты меня понимаешь, милый мой.', 'Дай, сделай милость!']), ('«Отрывок»', ['Действительный статский советник Козерогов, выйдя в отставку, купил себе небольшое имение и поселился в нем.', 'Здесь, подражая отчасти Цинциннату, отчасти же профессору Кайгородову, он трудился в поте лица и записывал свои наблюдения над природой.', 'После его смерти записки его вместе с прочим имуществом перешли по завещанию к его экономке Марфе Евлампиевне.', 'Как известно, почтенная старушка снесла барскую усадьбу и на месте ее построила превосходный трактир с продажею крепких напитков.', 'В этом трактире была особая «чистая» комната для проезжающих помещиков и чиновников, и на столе в комнате были положены записки покойного на случай, буде кому из проезжающих понадобится бумага.']), ('«Палат №6»', ['Всех их здесь пять человек.', 'Только один благородного звания, остальные же все мещане.', 'Первый от двери, высокий, худощавый мещанин с рыжими, блестящими усами и с заплаканными глазами, сидит, подперев голову, и глядит в одну точку.', 'День и ночь он грустит, покачивая головой, вздыхая и горько улыбаясь; в разговорах он редко принимает участие и на вопросы обыкновенно не отвечает.', 'Ест и пьет он машинально, когда дают.', 'Судя по мучительному, бьющему кашлю, худобе и румянцу на щеках, у него начинается чахотка.', 'За ним следует маленький, живой, очень подвижной старик с острою бородкой и с черными, кудрявыми, как у негра, волосами.', 'Днем он прогуливается по палате от окна к окну или сидит на своей постели, поджав по-турецки ноги, и неугомонно, как снегирь, насвистывает, тихо поет и хихикает.', 'Детскую веселость и живой характер проявляет он и ночью, когда встает за тем, чтобы помолиться богу, то есть постучать себя кулаками по груди и поковырять пальцем в дверях.', 'Это жид Мойсейка, дурачок, помешавшийся лет двадцать назад, когда у него сгорела шапочная мастерская.']), ('«Папаша»', ['Мамаша взвизгнула и жестом взбешенного трагика указала на портьеру... Папаша сконфузился, растерялся, ни к селу ни к городу запел какую-то песню и сбросил с себя сюртук... Он всегда терялся и становился совершенным идиотом, когда мамаша указывала ему на его портьеру.', 'Он сдался.', 'Позвали сына и потребовали от него слова.', 'Сынок рассердился, нахмурился, насупился и сказал, что он арифметику знает лучше самого учителя и что он не виноват в том, что на этом свете пятерки получаются одними только гимназистками, богачами да подлипалами.', 'Он разрыдался и сообщил адрес учителя арифметики во всех подробностях.', 'Папаша побрился, поводил у себя по лысине гребнем, оделся поприличнее и отправился «пожалеть единственного сына».']), ('«Пари»', ['Далее вспоминал он о том, что произошло после описанного вечера.', 'Решено было, что юрист будет отбывать свое заключение под строжайшим надзором в одном из флигелей, построенных в саду банкира.', 'Условились, что в продолжение пятнадцати лет он будет лишен права переступать порог флигеля, видеть живых людей, слышать человеческие голоса и получать письма и газеты.', 'Ему разрешалось иметь музыкальный инструмент, читать книги, писать письма, пить вино и курить табак.', 'С внешним миром, по условию, он мог сноситься не иначе, как молча, через маленькое окно, нарочно устроенное для этого.', 'Всё, что нужно, книги, ноты, вино и прочее, он мог получать по записке в каком угодно количестве, но только через окно.']), ('«Перед свадьбой»', ['Господин Назарьев — мужчина роста среднего, лицо имеет белое, ничего не выражающее, волосы курчавые, затылок плоский.', 'Где-то служит, жалованье получает тщедушное, едва на табак хватающее; вечно пахнет яичным мылом и карболкой, считает себя страшным волокитой, говорит громко, день и ночь удивляется; когда говорит — брызжет.', 'Франтит, на родителей смотрит свысока и ни одну барышню не пропустит, чтобы не сказать ей: «Как вы наивны!', 'Вы бы читали литературу!» Любит больше всего на свете свой почерк, журнал «Развлечение» и сапоги со скрипом, а наиболее всего самого себя, и в особенности в ту минуту, когда сидит в обществе девиц, пьет чай внакладку и с остервенением отрицает чертей.']), ('«Переполох»', ['И, сказав еще что-то, мадам Кушкина зашуршала шлейфом и вышла.', 'Машенька обвела удивленными глазами свою комнату и, ничего не понимая, не зная, что думать, пожала плечами, похолодела от страха... Что Федосья Васильевна искала в ее сумке?', 'Если действительно, как она говорит, она нечаянно зацепила рукавом и рассыпала, то зачем же выскочил из комнаты такой красный и взволнованный Николай Сергеич?', 'Зачем у стола слегка выдвинут один ящик?', 'Копилка, в которую гувернантка прятала гривенники и старые марки, была отперта.', 'Ее отперли, но запереть не сумели, хотя и исцарапали весь замок.', 'Этажерка с книгами, поверхность стола, постель — всё носило на себе свежие следы обыска.', 'И в корзине с бельем тоже.', 'Белье было сложено аккуратно, но не в том порядке, в каком оставила его Машенька, уходя из дому.']), ('«Пересолил»', ['Когда телега выехала со станции, были сумерки.', 'Направо от землемера тянулась темная, замерзшая равнина, без конца и краю... Поедешь по ней, так наверно заедешь к чёрту на кулички.', 'На горизонте, где она исчезала и сливалась с небом, лениво догорала холодная осенняя заря... Налево от дороги в темнеющем воздухе высились какие-то бугры, не то прошлогодние стоги, не то деревня.', 'Что было впереди, землемер не видел, ибо с этой стороны всё поле зрения застилала широкая, неуклюжая спина возницы.', 'Было тихо, но холодно, морозно.', '«Какая, однако, здесь глушь!', '— думал землемер, стараясь прикрыть свои уши воротником от шинели.', '— Ни кола ни двора.', 'Не ровен час — нападут и ограбят, так никто и не узнает, хоть из пушек пали... Да и возница ненадежный... Ишь, какая спинища!', 'Этакое дитя природы пальцем тронет, так душа вон!', 'И морда у него зверская, подозрительная».']), ('«Петров день»', ['Согласно предначертанному плану действий, решено было ехать прежде всего на крестьянский сенокос, находящийся в семи верстах от имения Егора Егорыча, — ехать на перепелов.', 'Приехавши на сенокос, охотники вылезли из тарантасов и разделились на две группы.', 'Одна группа, имея во главе генерала и Егора Егорыча, направилась направо; другая, с Кардамоновым во главе, пошла налево.', 'Больва отстал и пошел сам по себе.', 'На охоте он любил тишину и молчание.', 'Музыкант с лаем побежал вперед и через минуту согнал перепела.', 'Ваня выстрелил и не попал.', '— Высоко взял, чёрт возьми!', '— проворчал он.', 'Щенок Тщетный, взятый «приучаться», услышав первый раз в жизни выстрел, залаял и, поджав хвост, побежал к тарантасам.', 'Манже выстрелил в жаворонка и попал.']), ('«Попрыгунья»', ['— Как это могло вдруг случиться?', 'Вы слушайте, слушайте... Надо вам сказать, что отец служил вместе с Дымовым в одной больнице.', 'Когда бедняжка-отец заболел, то Дымов по целым дням и ночам дежурил около его постели.', 'Столько самопожертвования!', 'Слушайте, Рябовский... И вы, писатель, слушайте, это очень интересно.', 'Подойдите поближе.', 'Сколько самопожертвования, искреннего участия!', 'Я тоже не спала ночи и сидела около отца, и вдруг — здравствуйте, победила добра молодца!', 'Мой Дымов врезался по самые уши.', 'Право, судьба бывает так причудлива.', 'Ну, после смерти отца он иногда бывал у меня, встречался на улице и в один прекрасный вечер вдруг — бац!', 'сделал предложение... как снег на голову...']), ('«После театра»', ['Надя Зеленина, вернувшись с мамой из театра, где давали «Евгения Онегина», и придя к себе в комнату, быстро сбросила платье, распустила косу и в одной юбке и в белой кофточке поскорее села за стол, чтобы написать такое письмо, как Татьяна.', '«Я люблю вас, — написала она, — но вы меня не любите, не любите!» Написала и засмеялась.', 'Ей было только шестнадцать лет, и она еще никого не любила.', 'Она знала, что ее любят офицер Горный и студент Груздев, но теперь, после оперы, ей хотелось сомневаться в их любви.', 'Быть нелюбимой и несчастной — как это интересно!', 'В том, когда один любит больше, а другой равнодушен, есть что-то красивое, трогательное и поэтическое.', 'Онегин интересен тем, что совсем не любит, а Татьяна очаровательна, потому что очень любит, и если бы они одинаково любили друг друга и были счастливы, то, пожалуй, показались бы скучными.']), ('«Припадок»', ['Падших женщин он знал только понаслышке и из книг, и в тех домах, где они живут, не был ни разу в жизни.', 'Он знал, что есть такие безнравственные женщины, которые под давлением роковых обстоятельств — среды, дурного воспитания, нужды и т. п. вынуждены бывают продавать за деньги свою честь.', 'Они не знают чистой любви, не имеют детей, не правоспособны; матери и сестры оплакивают их, как мертвых, наука третирует их, как зло, мужчины говорят им ты.', 'Но, несмотря на всё это, они не теряют образа и подобия божия.', 'Все они сознают свой грех и надеются на спасение.', 'Средствами, которые ведут к спасению, они могут пользоваться в самых широких размерах.']), ('«Рассказ неизвестного человека»', ['Обыкновенно часов в одиннадцать утра в моей лакейской трещал электрический звонок, давая мне знать, что проснулся барин.', 'Когда я с вычищенным платьем и сапогами приходил в спальню, Георгий Иваныч сидел неподвижно в постели, не заспанный, а скорее утомленный сном, и глядел в одну точку, не выказывая по поводу своего пробуждения никакого удовольствия.', 'Я помогал ему одеваться, а он неохотно подчинялся мне, молча и не замечая моего присутствия; потом, с мокрою от умыванья головой и пахнущий свежими духами, он шел в столовую пить кофе.', 'Он сидел за столом, пил кофе и перелистывал газеты, а я и горничная Поля почтительно стояли у двери и смотрели на него.']), ('«Речь и ремешок»', ['Говорил он в таком роде очень долго.', 'Пронял всех нас, пронял теперешнее направление, похвалил науки и искусства, с оговоркой о пределе и рамках, из коих наукам выходить нельзя, и упомянул о любви матерей... Мы бледнели, краснели и слушали.', 'Душа наша мылась в его словах.', 'Нам хотелось умереть от раскаяния.', 'Нам хотелось облобызать его, пасть ниц... зарыдать... Я глядел в спину архивариуса, и мне казалось, что эта спина не плачет только потому, что боится нарушить общественную тишину.', 'Он полез в задний карман и вместе с платком вытащил оттуда какой-то ремешок.', 'Нечаянно, разумеется.', 'Ремешок, маленький, грязненький, закорузлый, поболтался в воздухе змейкой и упал к ногам архивариуса.', 'Архивариус поднял его обеими руками и с почтительным содроганием во всех членах положил на стол.']), ('«Розовый чулок»', ['В кабинете Сомов не один.', 'За его письменным столом сидит m-me Сомова, маленькая, хорошенькая дамочка в легкой блузе и в розовых чулочках.', 'Она усердно строчит письмо.', 'Проходя мимо нее, шагающий Иван Петрович всякий раз засматривает через ее плечо на писанье.', 'Он видит крупные хромающие буквы, узкие и тощие, с невозможными хвостами и закорючками.', 'Клякс, помарок и следов от пальцев многое множество.', 'Переносов m-me Сомова не любит, и каждая строка ее, дойдя до края листка, со страшными корчами, водопадом падает вниз... Сомов берет исписанные листки и, продолжая шагать, принимается за чтение.', 'Лидочка облокачивается о спинку кресла и следит за выражением его лица.', 'После первой же странички лицо его вытягивается и выражает что-то похожее на оторопь...']), ('«Роман с контрабасом»', ['Не долго думая, он разделся и погрузил свое тело в прохладные струи.', 'Вечер был великолепный.', 'Поэтическая душа Смычкова стала настраиваться соответственно гармонии окружающего.', 'Но какое сладкое чувство охватило его душу, когда, отплыв шагов на сто в сторону, он увидел красивую девушку, сидевшую на крутом берегу и удившую рыбу.', 'Он притаил дыхание и замер от наплыва разнородных чувств: воспоминания детства, тоска о минувшем, проснувшаяся любовь... Боже, а ведь он думал, что он уже не в состоянии любить!', 'После того, как он потерял веру в человечество (его горячо любимая жена бежала с его другом, фаготом Собакиным), грудь его наполнилась чувством пустоты, и он стал мизантропом.']), ('«Рыбья любовь»', ['Но далее.', 'На литейном заводе «Кранделя сыновья» служит директором инженер Крысин.', 'У него есть племянник Иван, который, как известно, пишет стихи и с жадностью печатает их во всех журналах и газетах.', 'В один знойный полдень молодой поэт, проходя мимо пруда, вздумал выкупаться.', 'Он разделся и полез в пруд.', 'Безумный карась принял его за Соню Мамочкину, подплыл к нему и нежно поцеловал его в спину.', 'Этот поцелуй имел самые гибельные последствия: карась заразил поэта пессимизмом.', 'Ничего не подозревая, поэт вылез из воды и, дико хохоча, отправился домой.', 'Через несколько дней он поехал в Петербург; побыв там в редакциях, он заразил всех поэтов пессимизмом, и с того времени наши поэты стали писать мрачные, унылые стихи.']), ('«Свадьба»', ['Наступает тишина.', 'Всем вдруг становится грустно.', 'Отец невесты, отставной подполковник, с тощим, испитым лицом, чувствуя, вероятно, что его куцая военная фигурка в рейтузах недостаточно торжественна, солидно надувает щеки и выпрямляется.', 'Он берет со столика образ.', 'Его жена, маленькая старушка в тюлевом чепце с широкими лентами, берет хлеб-соль и становится рядом с ним.', 'Начинается благословение.', 'Невеста Любочка бесшумно, как тень, опускается перед отцом на колени, и ее фата волнуется при этом и цепляется за цветы, разбросанные по платью, и из прически выбивается несколько шпилек.', 'Поклонившись образу и поцеловавшись с отцом, который еще сильнее надувает щеки, Любочка опускается перед матерью; фата ее опять цепляется, и две барышни, взволнованные, подбегают к ней, обдергивают, поправляют, прикалывают булавками...']), ('«Свирель»', ['Разморенный духотою еловой чащи, весь в паутине и в хвойных иглах, пробирался с ружьем к опушке приказчик из Дементьева хутора, Мелитон Шишкин.', 'Его Дамка — помесь дворняги с сеттером — необыкновенно худая и беременная, поджимая под себя мокрый хвост, плелась за хозяином и всячески старалась не наколоть себе носа.', 'Утро было нехорошее, пасмурное.', 'С деревьев, окутанных легким туманом, и с папоротника сыпались крупные брызги, лесная сырость издавала острый запах гнили.', 'Впереди, где кончалась чаща, стояли березы, а сквозь их стволы и ветви видна была туманная даль.', 'Кто-то за березами играл на самоделковой, пастушеской свирели.', 'Игрок брал не более пяти-шести нот, лениво тянул их, не стараясь связать их в мотив, но тем не менее в его писке слышалось что-то суровое и чрезвычайно тоскливое.']), ('«Святою ночью»', ['Погода казалась мне великолепной.', 'Было темно, но я все-таки видел и деревья, и воду, и людей... Мир освещался звездами, которые всплошную усыпали всё небо.', 'Не помню, когда в другое время я видел столько звезд.', 'Буквально некуда было пальцем ткнуть.', 'Тут были крупные, как гусиное яйцо, и мелкие, с конопляное зерно... Ради праздничного парада вышли они на небо все до одной, от мала до велика, умытые, обновленные, радостные, и все до одной тихо шевелили своими лучами.', 'Небо отражалось в воде; звезды купались в темной глубине и дрожали вместе с легкой зыбью.', 'В воздухе было тепло и тихо... Далеко, на том берегу, в непроглядной тьме, горело врассыпную несколько ярко-красных огней...']), ('«Сирена»', ['— Все мы сейчас желаем кушать, потому что утомились и уже четвертый час, но это, душа моя Григорий Саввич, не настоящий аппетит.', 'Настоящий, волчий аппетит, когда, кажется, отца родного съел бы, бывает только после физических движений, например, после охоты с гончими, или когда отмахаешь на обывательских верст сто без передышки.', 'Тоже много значит и воображение-с.', 'Ежели, положим, вы едете с охоты домой и желаете с аппетитом пообедать, то никогда не нужно думать об умном; умное да ученое всегда аппетит отшибает.', 'Сами изволите знать, философы и ученые насчет еды самые последние люди и хуже их, извините, не едят даже свиньи.', 'Едучи домой, надо стараться, чтобы голова думала только о графинчике да закусочке.']), ('«Скрипка Ротшильда»', ['Городок был маленький, хуже деревни, и жили в нем почти одни только старики, которые умирали так редко, что даже досадно.', 'В больницу же и в тюремный замок гробов требовалось очень мало.', 'Одним словом, дела были скверные.', 'Если бы Яков Иванов был гробовщиком в губернском городе, то, наверное, он имел бы собственный дом и звали бы его Яковом Матвеичем; здесь же в городишке звали его просто Яковом, уличное прозвище у него было почему-то Бронза, а жил он бедно, как простой мужик, в небольшом старой избе, где была одна только комната, и в этой комнате помещались он, Марфа, печь, двухспальная кровать, гробы, верстак и всё хозяйство.'])]

In [4]:
def natalie(text):
    text = re.sub('[\.,;]', '', text)  
    doc = Doc(text)
    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)
    text_raw = doc.tokens
    text_str = str(text_raw)
    text_str = re.sub('ADP', 'PREP', text_str)  # привожу к единому стандарту
    text_str = re.sub('AUX', 'VERB', text_str)
    text_str = re.sub('CCONJ', 'CONJ', text_str)
    text_str = re.sub('SCONJ', 'CONJ', text_str)
    text_str = re.sub('PRON', 'PRO', text_str)
    text_str = re.sub('PART', 'PTCL', text_str)
    text_str = re.findall('text=\'([а-яА-Яё]+)\',\spos=\'([A-Z]+)', text_str)
    return text_str


def lemmatiz(text):
    text = re.sub('[\.,;]', '', text)  
    doc_lemm = Doc(text)
    doc_lemm.segment(segmenter)
    doc_lemm.tag_morph(morph_tagger)
    text = doc_lemm.tokens
    for token in doc_lemm.tokens:
        token.lemmatize(morph_vocab)
    lemm_dict = {_.text: _.lemma for _ in doc_lemm.tokens}
    lemms = list(lemm_dict.items())
    return lemms

In [5]:
for i in range(len(corpus)):
    for k in range(len(corpus[i][1])):
        corpus_lower = corpus[i][1][k].lower()
        corpus_pos = natalie(corpus_lower)
        cor_str = corpus[i][1][k]
        corpus[i][1][k] = corpus[i][1][k].split()
        corpus[i][1][k] = (cor_str, corpus_pos)

In [6]:
print(corpus)
for a in range(len(corpus)):
    for b in range(len(corpus[a][1])):
        for c in range(len(corpus[a][1][b])):
            for d in range(len((corpus[a][1][b][1]))):
                e = lemmatiz(corpus[a][1][b][1][d][0])
                corpus[a][1][b][1][d] = (e[0][1], corpus[a][1][b][1][d][1])

print(corpus)

[('«Агафья»', [('Случилось мне быть у этого самого Савки в один из хороших майских вечеров.', [('случилось', 'VERB'), ('мне', 'PRO'), ('быть', 'VERB'), ('у', 'PREP'), ('этого', 'DET'), ('самого', 'ADJ'), ('савки', 'NOUN'), ('в', 'PREP'), ('один', 'NUM'), ('из', 'PREP'), ('хороших', 'ADJ'), ('майских', 'ADJ'), ('вечеров', 'NOUN')]), ('Помню, я лежал на рваной, затасканной полости почти у самого шалаша, от которого шел густой и душный запах сухих трав.', [('помню', 'VERB'), ('я', 'PRO'), ('лежал', 'VERB'), ('на', 'PREP'), ('рваной', 'ADJ'), ('затасканной', 'ADJ'), ('полости', 'NOUN'), ('почти', 'ADV'), ('у', 'PREP'), ('самого', 'ADJ'), ('шалаша', 'NOUN'), ('от', 'PREP'), ('которого', 'PRO'), ('шел', 'VERB'), ('густой', 'ADJ'), ('и', 'CONJ'), ('душный', 'ADJ'), ('запах', 'NOUN'), ('сухих', 'ADJ'), ('трав', 'NOUN')]), ('Подложив руки под голову, я глядел вперед себя.', [('подложив', 'VERB'), ('руки', 'NOUN'), ('под', 'PREP'), ('голову', 'NOUN'), ('я', 'PRO'), ('глядел', 'VERB'), ('вперед',

[('«Агафья»', [('Случилось мне быть у этого самого Савки в один из хороших майских вечеров.', [('случиться', 'VERB'), ('я', 'PRO'), ('быть', 'VERB'), ('у', 'PREP'), ('это', 'DET'), ('самый', 'ADJ'), ('савка', 'NOUN'), ('в', 'PREP'), ('один', 'NUM'), ('из', 'PREP'), ('хороший', 'ADJ'), ('майский', 'ADJ'), ('вечер', 'NOUN')]), ('Помню, я лежал на рваной, затасканной полости почти у самого шалаша, от которого шел густой и душный запах сухих трав.', [('помнить', 'VERB'), ('я', 'PRO'), ('лежать', 'VERB'), ('на', 'PREP'), ('рваный', 'ADJ'), ('затаскать', 'ADJ'), ('полость', 'NOUN'), ('почти', 'ADV'), ('у', 'PREP'), ('самый', 'ADJ'), ('шалаш', 'NOUN'), ('от', 'PREP'), ('который', 'PRO'), ('идти', 'VERB'), ('густой', 'ADJ'), ('и', 'CONJ'), ('душный', 'ADJ'), ('запах', 'NOUN'), ('сухой', 'ADJ'), ('трава', 'NOUN')]), ('Подложив руки под голову, я глядел вперед себя.', [('подложив', 'VERB'), ('рука', 'NOUN'), ('под', 'PREP'), ('голова', 'NOUN'), ('я', 'PRO'), ('глядеть', 'VERB'), ('вперед', 'ADV'

In [ ]:
print('запрос состоит из 1 элемента. формы запросов: «слово», «"слово"», «слово+POS»') 
search = input('Введите запрос:') 
search_split = search.split()
if len(search_split) > 1:
    print('запрос может состоять из максимум 1 элемента :)')
search_res = []
search_thisform = re.findall('^"', search)
if len(search_thisform) == 1:
    for i in corpus:
        for j in i[1]:
            search_new = re.sub('"', '', search)  
            line = search_new
            line += '\\b'
            check_form = re.findall(line, j[0])
            if len((check_form)) == 1:
                search_res.append((j[0], j[1], [i][0][0]))
                print(j[0], [i][0][0])
            else:
                pass
#print(search_res)

search_wordpos = re.findall('[а-яА-Я]+\+[A-Z]+', search)
if len(search_wordpos) == 1:
    for i in corpus:
        for j in i[1]:
            for h in range(len(j[1])):
                search_new = re.sub('"', '', search) 
                check_new = search_new.split('+')
                check_new_new = re.sub('[\[\]]', '',str(check_new))
                check_line = str(j[1][h])
                check_form = re.findall(check_new_new, check_line)
                #print(check_form)
                if len((check_form)) == 1:
                    search_res.append((j[0], j[1]))
                    print(j[0], [i][0][0])

                else:
                    pass
#print(search_res)

search_lemmatized = morph.parse(search)[0]
search_lemmatized = search_lemmatized.normal_form
for i in corpus:
    for j in i[1]:
        for h in j[:]:
            for g in h:
                for k in g:
                    l = search_lemmatized
                    l += '\\b'
                    check_form = re.findall(l, k)
                    if len((check_form)) == 1:
                        #print(j[0])
                        search_res.append((j[0], j[1]))
                        print(j[0], [i][0][0])
                    else:
                        pass
                    
#print(search_res)
if len(search_res)  < 1:
  print('ничего не найдено :( попробуйте другой')